In [10]:
import codecs
import os
from collections import defaultdict

input_path = 'C:\\Users\\user\\Desktop\\RF result\\顯著特徵尋找\\特徵值\\'

classification_name = ['雷震','殷海光','夏道平','傅正','龍平甫','蔣勻田','朱伴耘','胡適','羅鴻詔']
feature_condicate = ['高頻','bigram','trigram','標點','N+N','N+V','VH+N','D+V','否定','程度','情態']

for file in os.listdir(input_path):
    with codecs.open(input_path+file,'rb','utf8') as f:
        
        if file.split('.')[0].split()[1] not in feature_condicate:
            continue
            
        print (file.split('.')[0])
        content = f.readlines()
        
        all_feature = []
        
        feature_importance = defaultdict(list)
        name = ''
        count = 2
        for line in content:
            if line[0] == '#':
                name = line[1:].strip()
                continue
            elif name != '' and '雷震' not in name:
                continue
            elif count > 0:
                count = count - 1
                continue
            elif line.strip() == '':
                count = 2
                continue
            else:
                temp_line = line.strip().split()
                if len(temp_line) != 2:
                    continue
                if float(temp_line[1]) > 0.01:
                    feature_importance[name].append((temp_line[0],float(temp_line[1])))
                    if temp_line[0] not in all_feature:
                        all_feature.append(temp_line[0])

        feature_num = defaultdict(int)
        for i in all_feature:
            for k,v in feature_importance.items():
                check = True
                for index,element in enumerate(v):
                    if element[0] == i:
                        feature_num[i] += (index + 1)
                        check = False
                if check:
                    feature_num[i] += 100 #懲罰數值
                        
        so = sorted(feature_num.items(), key=lambda d:d[1], reverse = False)
        for i,e in enumerate(so):
            if i >= 10:
                break
            print (e[0],e[1])
        print ()

FC bigram name
我們的 320
都是 416
這一 417
民主國家 428
而言 510
這個 515
是在 605
這樣的 607
他的 609
的思想 610

FC D+V name
不(D)敢(VL) 21
可以(D)說(VE) 32
不(D)願(VK) 511
可(D)言(VE) 512
很(Dfa)明顯(VH) 604
最(Dfa)重要(VH) 608
不(D)必要(VH) 608
最(Dfa)大(VH) 608
不(D)知(VK) 610
所(D)說(VE) 610

FC N+N name
民主(Na)國家(Na) 21
民主(Na)政治(Na) 223
民意(Na)機關(Na) 318
司法(Na)機關(Na) 333
人民(Na)代表(Na) 417
臺灣(Nc)省政府(Nc) 425
政府(Na)當局(Na) 429
帝國(Na)主義(Na) 509
主義(Na)者(Na) 514
世界(Nc)大戰(Na) 517

FC N+V name
人(Na)說(VE) 223
自由(Na)民主(VH) 238
言論(Na)自由(VH) 325
人(Na)以為(VE) 339
民主(Na)自由(VH) 412
世界(Nc)和平(VH) 517
黨(Na)專政(VA) 520
世界(Nc)革命(VA) 603
才能(Na)使(VL) 607
人(Na)懷疑(VK) 612

FC trigram name
是一個 412
並不是 415
有一個 417
也不是 509
所說的 608
的一個 701
各國的 701
這句話 701
自由世界的 701
這一套 701

FC VH+N name
自由(VH)中國(Nc) 9
民主(VH)政治(Na) 20
民主(VH)國家(Na) 28
新(VH)憲法(Na) 230
實際(VH)政治(Na) 233
自由(VH)思想(Na) 326
大(VH)原因(Na) 432
先進(VH)國家(Na) 433
自由(VH)世界(Nc) 604
反共(VH)人士(Na) 614

FC 否定 name
不(D)是(SHI) 9
不(D)敢(VL) 117
不(D)應(D) 125
不(D)知(VK) 325
不(D)願(VK) 420
不(D)在(P) 609
不(D)以(P) 609
不(D)出(V

In [65]:
import codecs
import os
from collections import defaultdict
import time
import numpy as np
from pandas import Series, DataFrame 

input_path = 'C:\\Users\\user\\Desktop\\RF result\\顯著特徵尋找\\特徵值\\'

classification_name = ['雷震','殷海光','夏道平','傅正','龍平甫','蔣勻田','朱伴耘','胡適','羅鴻詔']
classification_topic = ['社論','文章','日記']
feature_condicate = ['高頻','bigram','trigram','標點','N+N','N+V','VH+N','D+V','否定','程度','情態']

name_index = defaultdict(int)
for index,name in enumerate(classification_name): 
    name_index[name] = index
    
topic_index = defaultdict(int)
for index,topic in enumerate(classification_topic): 
    topic_index[topic] = index

for file in os.listdir(input_path):
    with codecs.open(input_path+file,'rb','utf8') as f:
        content = f.readlines()
        
        print (file)
        
        temp_label = ''
        temp_index = ''
        if 'name' in file:
            temp_label = classification_name
            temp_index = name_index
        elif 'topic' in file:
            temp_label = classification_topic
            temp_index = topic_index
        
        accuracy = []
        vector_space = np.zeros((len(temp_index)+1,len(temp_index)+1),np.float64) #多平均
        
        for i in range(len(content)-1):
            if content[i][0] == '#':
                accuracy.append((content[i][1:],float(content[i+1].split('：')[1])))
                
        for i in accuracy:
            #print (i[0],round(i[1],3))
            vector_space[temp_index[i[0].split()[0]],temp_index[i[0].split()[1]]] = round(i[1],3)
            vector_space[temp_index[i[0].split()[1]],temp_index[i[0].split()[0]]] = round(i[1],3)
            
        for i in range(len(temp_index)):
            vector_space[i,-1] = round(sum(vector_space[i])/(len(temp_index)-1),3)
        for i in range(len(temp_index)):
            vector_space[-1,i] = round(sum(vector_space[:,i])/(len(temp_index)-1),3)
        vector_space[-1,-1] = round(sum(vector_space[-1])/(len(temp_index)),3)
        
        print (' '.join(temp_label+['平均']))
        print (vector_space)
        #df = DataFrame(vector_space,index=temp_label,columns=temp_label)
        print ()

FC bigram name.txt
雷震 殷海光 夏道平 傅正 龍平甫 蔣勻田 朱伴耘 胡適 羅鴻詔 平均
[[ 0.     0.917  0.786  0.8    1.     1.     1.     0.833  1.     0.917]
 [ 0.917  0.     0.923  1.     1.     1.     1.     0.917  1.     0.97 ]
 [ 0.786  0.923  0.     1.     1.     1.     0.929  0.714  1.     0.919]
 [ 0.8    1.     1.     0.     1.     1.     1.     1.     1.     0.975]
 [ 1.     1.     1.     1.     0.     0.933  1.     0.933  0.857  0.965]
 [ 1.     1.     1.     1.     0.933  0.     1.     0.769  1.     0.963]
 [ 1.     1.     0.929  1.     1.     1.     0.     1.     1.     0.991]
 [ 0.833  0.917  0.714  1.     0.933  0.769  1.     0.     0.917  0.885]
 [ 1.     1.     1.     1.     0.857  1.     1.     0.917  0.     0.972]
 [ 0.917  0.97   0.919  0.975  0.965  0.963  0.991  0.885  0.972  0.951]]

FC D+V name.txt
雷震 殷海光 夏道平 傅正 龍平甫 蔣勻田 朱伴耘 胡適 羅鴻詔 平均
[[ 0.     0.583  0.643  0.6    0.714  0.5    0.833  0.583  0.833  0.661]
 [ 0.583  0.     0.846  0.8    0.929  0.667  1.     1.     0.818  0.83 ]
 [ 0.643  0.846